# Exp B2: Anti-Aligned Curve - True Negative Control

## 目的
「価値勾配が逆方向なら性能が悪化する」を正しく検証する

### exp_B（Uninformative）との違い
- **exp_B**: random labels → 直交/正則化効果
- **exp_B2**: anti labels `(y+1) mod 10` → 系統的に逆方向

### 実験条件
- λ: 0.1, 0.2, 0.3, 0.4, 0.5, 0.6 (6点)
- Noise: 40%, 80%
- Seeds: 0, 1, 2
- Method: **Anti-aligned** (systematic wrong labels)

### runs数: 36 runs
### 推定時間: ~5.2時間

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
SAVE_DIR = '/content/drive/MyDrive/dual-gradient-results/exp_B2_anti_aligned'
os.makedirs(SAVE_DIR, exist_ok=True)
print(f'Save directory: {SAVE_DIR}')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
import numpy as np
import json
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    print(f'GPU: {torch.cuda.get_device_name(0)}')
print(f'Device: {device}')

In [ ]:
class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, 1, stride, bias=False),
                nn.BatchNorm2d(planes))
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return F.relu(out)

class ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, 3, 1, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(64, 2, 1)
        self.layer2 = self._make_layer(128, 2, 2)
        self.layer3 = self._make_layer(256, 2, 2)
        self.layer4 = self._make_layer(512, 2, 2)
        self.linear = nn.Linear(512, num_classes)
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for s in strides:
            layers.append(BasicBlock(self.in_planes, planes, s))
            self.in_planes = planes
        return nn.Sequential(*layers)
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.adaptive_avg_pool2d(out, 1)
        out = out.view(out.size(0), -1)
        return self.linear(out)

In [ ]:
BATCH_SIZE = 256
NUM_WORKERS = 4
EPOCHS = 100
LR = 0.1

LAMBDAS = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
NOISE_RATES = [0.4, 0.8]
SEEDS = [0, 1, 2]

total_runs = len(LAMBDAS) * len(NOISE_RATES) * len(SEEDS)
print(f'Total runs: {total_runs}')
print(f'Estimated time: {total_runs * 9.2 / 60:.1f} hours')

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

def load_cifar10():
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    return trainset, testset

def get_data_loaders(trainset, testset):
    indexed_trainset = IndexedDataset(trainset)
    train_loader = DataLoader(indexed_trainset, batch_size=BATCH_SIZE, shuffle=True,
                              num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True, drop_last=True)
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False,
                             num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True)
    return train_loader, test_loader

def inject_noise(labels, noise_rate, seed):
    np.random.seed(seed + 1000)
    noisy_labels = labels.copy()
    n_noisy = int(noise_rate * len(labels))
    noisy_indices = np.random.choice(len(labels), n_noisy, replace=False)
    for idx in noisy_indices:
        choices = [i for i in range(10) if i != labels[idx]]
        noisy_labels[idx] = np.random.choice(choices)
    return noisy_labels

def create_anti_labels(clean_labels):
    """★ Anti-aligned: 系統的に間違ったラベル (y+1) mod 10"""
    return (clean_labels + 1) % 10

In [ ]:
def evaluate(model, test_loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device, non_blocking=True), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total

def train_dual_gradient_anti_aligned(model, train_loader, test_loader, clean_labels, noisy_labels, lam):
    """★ Anti-aligned版: 系統的に逆方向のラベルを使用"""
    optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 75], gamma=0.1)
    criterion = nn.CrossEntropyLoss()
    
    noisy_labels_tensor = torch.tensor(noisy_labels, device=device)
    
    # ★★★ Anti-aligned: (y+1) mod 10 ★★★
    anti_labels = create_anti_labels(clean_labels)
    anti_labels_tensor = torch.tensor(anti_labels, device=device)
    
    # 検証: anti_labelsが正しく生成されているか確認
    match_rate = (anti_labels == clean_labels).mean()
    print(f'    Anti-label match rate with clean: {match_rate:.4f} (should be 0.0)')
    
    K, cached_value_grad, global_step, best_acc = 16, None, 0, 0
    
    for epoch in range(EPOCHS):
        model.train()
        for inputs, _, indices in train_loader:
            inputs = inputs.to(device, non_blocking=True)
            indices = indices.to(device, non_blocking=True)
            batch_noisy = noisy_labels_tensor[indices]
            batch_anti = anti_labels_tensor[indices]  # ★ Anti-aligned!
            
            # 構造勾配（ノイズラベルから）
            optimizer.zero_grad()
            outputs = model(inputs)
            loss_struct = criterion(outputs, batch_noisy)
            loss_struct.backward(retain_graph=True)
            g_struct = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            # 価値勾配（Anti-alignedラベルから）★ ここが核心
            if global_step % K == 0 or cached_value_grad is None:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss_value = criterion(outputs, batch_anti)  # ★ Anti-aligned!
                loss_value.backward()
                cached_value_grad = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            # 勾配の正規化と混合
            g_struct_norm = g_struct / (g_struct.norm() + 1e-12)
            g_value_norm = cached_value_grad / (cached_value_grad.norm() + 1e-12)
            g_mix = (1 - lam) * g_struct_norm + lam * g_value_norm
            
            # パラメータ更新
            optimizer.zero_grad()
            idx = 0
            for p in model.parameters():
                numel = p.numel()
                p.grad = g_mix[idx:idx+numel].view(p.shape).clone()
                idx += numel
            optimizer.step()
            global_step += 1
        scheduler.step()
        if (epoch + 1) % 10 == 0:
            best_acc = max(best_acc, evaluate(model, test_loader))
    final_acc = evaluate(model, test_loader)
    return final_acc, max(best_acc, final_acc)

In [ ]:
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader, test_loader = get_data_loaders(trainset, testset)

print('Verifying IndexedDataset...')
sample_batch = next(iter(train_loader))
print(f'  ✓ indices shape: {list(sample_batch[2].shape)}')

# Anti-labelsの検証
print('\nVerifying Anti-labels...')
anti_labels = create_anti_labels(clean_labels)
print(f'  Clean label examples: {clean_labels[:10]}')
print(f'  Anti label examples:  {anti_labels[:10]}')
print(f'  Match rate: {(anti_labels == clean_labels).mean():.4f} (should be 0.0)')

print('\nWarming up GPU...')
warmup_model = ResNet18().to(device)
for _ in range(20):
    _ = warmup_model(torch.randn(BATCH_SIZE, 3, 32, 32, device=device))
del warmup_model
torch.cuda.empty_cache()
print('Warmup complete.')

In [ ]:
results = []
checkpoint_file = f'{SAVE_DIR}/checkpoint.json'
completed = set()

if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        results = json.load(f)
    for r in results:
        completed.add((r['lambda'], r['noise_rate'], r['seed']))
    print(f'Checkpoint loaded: {len(completed)} runs')

run_counter = 0
exp_start = time.time()

for noise_rate in NOISE_RATES:
    print(f'\n{"="*60}')
    print(f'NOISE RATE: {int(noise_rate*100)}%')
    print(f'{"="*60}')
    
    for lam in LAMBDAS:
        for seed in SEEDS:
            run_counter += 1
            key = (lam, noise_rate, seed)
            
            if key in completed:
                print(f'[{run_counter}/{total_runs}] λ={lam} seed={seed} - SKIPPED')
                continue
            
            print(f'\n[{run_counter}/{total_runs}] ANTI-ALIGNED λ={lam} noise={noise_rate} seed={seed}')
            t0 = time.time()
            
            set_seed(seed)
            noisy_labels = inject_noise(clean_labels, noise_rate, seed)
            model = ResNet18().to(device)
            
            final_acc, best_acc = train_dual_gradient_anti_aligned(
                model, train_loader, test_loader, clean_labels, noisy_labels, lam)
            elapsed = time.time() - t0
            
            results.append({
                'experiment': 'exp_B2_anti_aligned',
                'method': 'Anti-aligned',
                'lambda': lam,
                'noise_rate': noise_rate,
                'seed': seed,
                'test_acc': final_acc,
                'test_error': 1 - final_acc,
                'best_test_error': 1 - best_acc,
                'time_seconds': elapsed
            })
            
            with open(checkpoint_file, 'w') as f:
                json.dump(results, f, indent=2)
            
            # 判定（CE baseline ≈ 38% at 40% noise, ≈ 62% at 80% noise）
            ce_baseline = 0.38 if noise_rate == 0.4 else 0.62
            if (1 - final_acc) > 0.8:
                status = ' ⚠️ COLLAPSE!'
            elif (1 - final_acc) > ce_baseline:
                status = ' ✓ DEGRADED (expected)'
            else:
                status = ' ⁉️ IMPROVED (unexpected!)'
            
            print(f'  Error: {1-final_acc:.4f} | Best: {1-best_acc:.4f} | Time: {elapsed/60:.1f} min{status}')

print(f'\n{"="*60}')
print(f'EXPERIMENT COMPLETE')
print(f'Total time: {(time.time()-exp_start)/3600:.2f} hours')
print(f'{"="*60}')

In [ ]:
import pandas as pd

with open(f'{SAVE_DIR}/exp_B2_results.json', 'w') as f:
    json.dump(results, f, indent=2)

df = pd.DataFrame(results)

print('\n' + '='*60)
print('SUMMARY')
print('='*60)

summary = df.groupby(['noise_rate', 'lambda']).agg({
    'test_error': ['mean', 'std', 'count']
}).round(4)
print(summary)
summary.to_csv(f'{SAVE_DIR}/exp_B2_summary.csv')

# 比較表
print('\n' + '='*60)
print('COMPARISON: Expected vs Actual')
print('='*60)
print('Expectation: Anti-aligned should DEGRADE performance')
print('CE baseline: ~38% (40% noise), ~62% (80% noise)')
print()
for noise in NOISE_RATES:
    ce_baseline = 0.38 if noise == 0.4 else 0.62
    print(f'Noise {int(noise*100)}%:')
    for lam in LAMBDAS:
        df_sub = df[(df['noise_rate'] == noise) & (df['lambda'] == lam)]
        error = df_sub['test_error'].mean()
        status = '✓ DEGRADED' if error > ce_baseline else '⁉️ IMPROVED'
        print(f'  λ={lam}: {error:.3f} ({status})')

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 6))

for noise_rate in NOISE_RATES:
    df_n = df[df['noise_rate'] == noise_rate]
    g = df_n.groupby('lambda')['test_error'].agg(['mean', 'std'])
    ax.errorbar(g.index, g['mean'], yerr=g['std'], marker='s', capsize=4, 
                linewidth=2, markersize=8, label=f'Anti-aligned (Noise {int(noise_rate*100)}%)')

# CE baselines
ax.axhline(y=0.38, color='blue', linestyle='--', alpha=0.5, label='CE baseline (40% noise)')
ax.axhline(y=0.62, color='red', linestyle='--', alpha=0.5, label='CE baseline (80% noise)')
ax.axhline(y=0.9, color='gray', linestyle=':', alpha=0.5, label='Random guess')

ax.set_xlabel('λ', fontsize=13)
ax.set_ylabel('Test Error', fontsize=13)
ax.set_title('Anti-Aligned Curve (y_anti = (y+1) mod 10)\nExpected: All points above CE baseline', fontsize=13)
ax.legend(loc='upper left')
ax.grid(alpha=0.3)
ax.set_ylim(0, 1)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/exp_B2_anti_aligned_plot.png', dpi=300, bbox_inches='tight')
plt.show()

print(f'Figure saved to: {SAVE_DIR}/exp_B2_anti_aligned_plot.png')